
-  Web Scraping: the automated process of extracting data from websites

In [ ]:
''' BLOQUE 1 - Scraping de todas las páginas del sitio'''

# importar dependencias 
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import json

# Se crea una lista vacía donde guardar todos los libros
libros_registrados = []

# Establecer la URL base
url_inicio = "https://books.toscrape.com/catalogue/page-1.html"
url_base = "http://books.toscrape.com/catalogue/"

next_page = url_inicio

# Mientras haya una página siguiente:
while next_page:

    # Hacer request a la página actual
    nuevo_request = requests.get(next_page)
    # Parsear el HTML
    nuevo_soup = BeautifulSoup(nuevo_request.text, 'lxml')

    # Buscar todos los libros de esa página
    libros = nuevo_soup.find_all('article', class_='product_pod')

    # Por cada libro, extraer: título, precio, disponibilidad, link
    for libro in libros:
        titulos = libro.find('h3').a['title']
        precios = libro.find('p', class_='price_color').text.strip()
        disponibilidad = libro.find('p', class_='instock availability').text.strip()
        a_links = libro.find('h3').a['href']
        link_completo = urljoin(next_page,a_links)

        # Guardar la info en la lista de libros
        libros_registrados.append({
        'Titulo': titulos,
        'Precio': precios,
        'Disponibilidad': disponibilidad,
        'Link': link_completo
        })
    
    # Buscar el enlace a la siguiente página (si existe)
    boton_next = nuevo_soup.find('li', class_='next')
    # condicion de boton next
    if boton_next:
        href_next = boton_next.a['href']
        # Construir la URL completa de la siguiente página, y que continue el bucle anterior
        next_page = urljoin(next_page, href_next)
    else:
        break

# Una vez fuera del bucle, tendremos una lista con todos los libros de todas las páginas
print(f'50 paginas scrapeadas con exito')
print(f'Vamos por los detalles de cada libro')

'''
BLOQUE 2.
Recorrer cada libro de libros_registrados y visitar su página individual para obtener más detalles que no aparecen en la página principal.
Ej: Rating, categoría, descripción, UPC, tipo de producto, disponibilidad, etc.
'''

# 1. recorrer cada libro dentro de la lista de los libros registrados
for libro in libros_registrados:
    # 2. visitar su pagina individual
    link_individual = libro['Link']
    link_request = requests.get(link_individual)
    link_individual_soup = BeautifulSoup(link_request.text, 'lxml')

    #specs = link_individual_soup.find('div', class_='page_inner')
    specs = link_individual_soup
    
    # 3. extraer detalles:

    ## rating
    rating_tag = specs.find('p', class_='star-rating')
    #print(rating_tag)
    if rating_tag:
        rating = rating_tag['class'][1]     # Ej: 'star-rating Five'
    else:
        rating = 'Sin rating'

    ## categoria
    breadcrumb = specs.find('ul', class_='breadcrumb')
    if breadcrumb:
        categoria = breadcrumb.find_all('li')[2].text.strip()
    else:
        categoria = 'Sin categoria'

    ## descripcion
    descripcion_tag = specs.find('div', id='product_description')
    #print(descripcion_tag)
    if descripcion_tag:
        descripcion = descripcion_tag.find_next_sibling('p').text.strip()
    else:
        descripcion = 'Sin descripcion'

    ## UPC (codigo universal del producto)
    tabla_informacion = specs.find('table', class_='table table-striped')
    if tabla_informacion:
        upc = tabla_informacion.find('tr').find('td').text 
    else:
        upc = 'Sin UPC' 

    # Actualiza el diccionario
    libro['Rating'] = rating
    libro['Descripcion'] = descripcion
    libro['Categoria'] = categoria
    libro['UPC'] = upc

print("Scraping completo. Todos los detalles fueron agregados.")

# Guardar en JSON
with open('libros_scrapeados.json', 'w', encoding='utf-8') as f:
    json.dump(libros_registrados, f, ensure_ascii=False, indent=2)

# Leer desde JSON
with open('libros_scrapeados.json', 'r', encoding='utf-8') as f:
    datos = json.load(f)
    print(f'Se guardaron {len(datos)} libros')

    # Mostrar el primero para validar estructura
    print(datos[0])  

# Abrir automáticamente el archivo (solo en Windows)
import os
os.startfile('libros_scrapeados.json')


50 paginas scrapeadas con exito
Vamos por los detalles de cada libro
Scraping completo. Todos los detalles fueron agregados.
Se guardaron 1000 libros
{'Titulo': 'A Light in the Attic', 'Precio': 'Â£51.77', 'Disponibilidad': 'In stock', 'Link': 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html', 'Rating': 'Three', 'Descripcion': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. 